# Lab | Inferential statistics - T-test & P-value

In [1]:
import pandas as pd
import numpy as np
import scipy.stats as st
from scipy.stats import f_oneway
import matplotlib.pyplot as plt
import seaborn as sns
import math
from pathlib import Path

1. One-tailed t-test - In a packing plant, a machine packs cartons with jars. It is supposed that a new machine will pack faster on average than the machine currently used. To test that hypothesis, the times each machine takes to pack ten cartons are recorded. The results are in seconds in the tables in the file files_for_lab/machine.txt. Assume that there is sufficient evidence to conduct the t-test, does the data provide sufficient evidence to show if one machine is better than the other?

In [2]:
data=pd.read_csv("machine.txt", encoding="utf_16_le", sep="\t")
data.head()

,New machine,Old machine
0,42.1,42.7
1,41.0,43.6
2,41.3,43.8
3,41.8,43.3
4,42.4,42.5


In [3]:
data = data.rename(columns={'    Old machine': 'Old machine'})


In [4]:
st.ttest_ind(data['New machine'], data['Old machine'], equal_var=False)

TtestResult(statistic=-3.3972307061176026, pvalue=0.00324224946631798, df=17.847298506720346)

In [6]:
# Check if we accept the null hypothesis
pvalue=0.00324224946631798
alpha = 0.05
if pvalue < alpha:
    print("Reject the null hypothesis. There is sufficient evidence to suggest that the new machine packs faster.")
else:
    print("Fail to reject the null hypothesis. There is not sufficient evidence to suggest that the new machine packs faster.")

Reject the null hypothesis. There is sufficient evidence to suggest that the new machine packs faster.


2. Matched Pairs Test - In this challenge we will compare dependent samples of data describing our Pokemon (file files_for_lab/pokemon.csv). Our goal is to see whether there is a significant difference between each Pokemon's defense and attack scores. Our hypothesis is that the defense and attack scores are equal. Compare the two columns to see if there is a statistically significant difference between them and comment on your result.

In [7]:
pokemon_data = pd.read_csv('pokemon.csv')
pokemon_data.head()

,#,Name,Type 1,Type 2,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary
0,1,Bulbasaur,Grass,Poison,318,45,49,49,65,65,45,1,False
1,2,Ivysaur,Grass,Poison,405,60,62,63,80,80,60,1,False
2,3,Venusaur,Grass,Poison,525,80,82,83,100,100,80,1,False
3,3,VenusaurMega Venusaur,Grass,Poison,625,80,100,123,122,120,80,1,False
4,4,Charmander,Fire,NaN,309,39,52,43,60,50,65,1,False


In [8]:
defense = pokemon_data['Defense']
attack = pokemon_data['Attack']

In [9]:
t_stat, p_value = st.ttest_ind(defense, attack)
print("t-statistic:", t_stat)
print("p-value:", p_value)


t-statistic: -3.241764074042312
p-value: 0.0012123980547321484


In [10]:
# Check if the p-value is less than the significance level (e.g., 0.05)
alpha = 0.05
if p_value < alpha:
    print("Reject the null hypothesis. There is sufficient evidence to suggest a significant difference between defense and attack scores.")
else:
    print("Fail to reject the null hypothesis. There is not sufficient evidence to suggest a significant difference between defense and attack scores.")

Reject the null hypothesis. There is sufficient evidence to suggest a significant difference between defense and attack scores.


Part 1
In this activity, we will look at another example. Your task is to understand the problem and write down all the steps to set up ANOVA. After the next lesson, we will ask you to solve this problem using Python. Here are the steps that you would need to work on: - Null hypothesis - Alternate hypothesis - Level of significance - Test statistic - P-value - F table

Context
Suppose you are working as an analyst in a microprocessor chip manufacturing plant. You have been given the task of analyzing a plasma etching process with respect to changing Power (in Watts) of the plasma beam. Data was collected and provided to you to conduct statistical analysis and check if changing the power of the plasma beam has any effect on the etching rate by the machine. You will conduct ANOVA and check if there is any difference in the mean etching rate for different levels of power. You can find the data anova_lab_data.xlsx file in the files_for_lab folder

State the null hypothesis
State the alternate hypothesis
What is the significance level
What are the degrees of freedom of the model, error terms, and total DoF
Data were collected randomly and provided to you in the table as shown: link to the image - Data

In [11]:
data = pd.read_excel('anova_lab_data.xlsx')
data

,Power,Etching Rate
0,160 W,5.43
1,180 W,6.24
2,200 W,8.79
3,160 W,5.71
4,180 W,6.71
5,200 W,9.20
6,160 W,6.22
7,180 W,5.98
8,200 W,7.90
9,160 W,6.01


In [12]:
data.columns

Index(['Power ', 'Etching Rate'], dtype='object')

In [13]:
data.columns = data.columns.str.replace(' ', '')
data.columns

Index(['Power', 'EtchingRate'], dtype='object')

In [17]:
data_result = st.f_oneway(data[data['Power'] == '160 W']['EtchingRate'],
                              data[data['Power'] == '180 W']['EtchingRate'],
                              data[data['Power'] == '200 W']['EtchingRate'])

data_result

F_onewayResult(statistic=36.87895470100505, pvalue=7.506584272358903e-06)

In [18]:
group_data = data.groupby('Power')['EtchingRate'].agg(Power_mean='mean',Samples='size').reset_index()
group_data

,Power,Power_mean,Samples
0,160 W,5.792,5
1,180 W,6.238,5
2,200 W,8.318,5


Using python

In [19]:
S2t = 0
for power in data['Power'].unique():
    ng = len(data[data['Power'] == power])
    S2t  += ng * ( ( data[data['Power'] == power]['EtchingRate'].mean() - data['EtchingRate'].mean() ) ** 2)
S2t /= ( data['Power'].nunique() - 1 ) 

print("The value of S2t is {:.2f}".format(S2t))

The value of S2t is 9.09


In [20]:
S2E = 0
for power in data['Power'].unique():
    for rate in data[data['Power'] == power]['EtchingRate']:
        S2E += ( rate - data[data['Power'] == power]['EtchingRate'].mean() ) ** 2
S2E /= ( len(data) - data['Power'].nunique() )

print()
print("The value of S2E is {:.2f}".format(S2E))


The value of S2E is 0.25


In [21]:
S2E2 = 0
for power in data['Power'].unique():
    ng = len(data[data['Power'] == power])
    S2E2 += ( ng - 1 ) * np.var(data[data['Power'] == power]['EtchingRate'],ddof=1)
S2E2 /= ( len(data) - data['Power'].nunique() )

print()
print("The value of S2E is {:.2f}".format(S2E2))


The value of S2E is 0.25


In [22]:
F = S2t / S2E
print("The value of F is {:.2f}".format(F))

The value of F is 36.88


In [23]:
df1 = data['Power'].nunique() - 1
df2 = len(data) - data['Power'].nunique()

print("Number of degrees of freedom df1: ",df1)
print("Number of degrees of freedom df2: ",df2)


Number of degrees of freedom df1:  2
Number of degrees of freedom df2:  12


In [24]:
st.f.cdf(F,dfn=df1, dfd=df2)

0.9999924934157276

In [25]:
1 - st.f.cdf(F,dfn=df1, dfd=df2)

7.5065842723986975e-06